# Travelling Salesman Problem

This is an homework of Artificial inteligence.

The homework is to code the travelling salesman problem using genetic algorithm.

Group:
- Hendy Rodrigues F. Silva

Id: 1510081

**Homework by Professor Aragão Junior**




In [49]:
coord_x_file = open("data/x.dat","r")
coord_y_file = open("data/y.dat","r")

coord_x = coord_x_file.readlines()
coord_x = list(map(lambda x: x.rstrip().strip() ,coord_x))

coord_y = coord_y_file.readlines()
coord_y = list(map(lambda x: int(x.strip()) ,coord_y))

print (coord_x)

['0', '42', '43', '33', '77', '71', '34', '8', '79', '31', '96', '87', '7', '96', '11', '94', '16', '5', '27', '14', '57', '76', '44', '66', '27', '33', '0', '76', '67', '65', '60', '51', '75', '44', '14', '62', '4', '62', '33', '99', '43', '57', '22', '28', '73', '86', '88', '11', '32', '29', '96', '49', '65', '79', '28', '46', '15', '53', '48', '10', '2', '48', '7', '28', '27', '20', '83', '65', '65', '61', '19', '16', '11', '44', '5', '87', '68', '24', '48', '62', '62', '38', '61', '6', '6', '79', '74', '91', '87', '19', '76', '88', '97', '68', '53', '94', '48', '57', '55', '72', '']
